## Library import 

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
import sys
from urllib.error import HTTPError

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [3]:
players_boxscores = pd.read_csv('./pipeline_output/nba_players_boxscores_2022-09-08.csv')
players_boxscores.shape
# # test.head()

(597659, 32)

In [4]:
players_boxscores.head()

,id_season,player_name,rk,game_id,game_date,age,tm,ext_dom,ppp,results,gs,mp,fg,fga,fg_prct%,3p,3pa,3p_prct,ft,fta,ft_prct,orb,drb,trb,ast,stl,blk,tov,pf,pts,player_game_score,plus_minus
0,2001,AC Green,1,1,2000-11-01,37-028,MIA,NaN,ORL,W (+26),0,14:53,2,4,0.500,0,0,NaN,1,2,0.5,1,3,4,0,0,1,1,2,5,3.1,15.0
1,2001,AC Green,2,2,2000-11-03,37-030,MIA,@,CHH,L (-4),0,7:08,0,1,0.000,0,0,NaN,0,0,NaN,0,1,1,0,0,0,3,1,0,-3.8,-10.0
2,2001,AC Green,3,3,2000-11-04,37-031,MIA,NaN,PHI,L (-2),0,5:37,1,3,0.333,0,0,NaN,0,0,NaN,1,0,1,0,0,0,0,2,2,0.2,4.0
3,2001,AC Green,4,4,2000-11-08,37-035,MIA,NaN,SEA,W (+6),0,13:21,0,3,0.000,0,0,NaN,0,0,NaN,0,1,1,0,0,0,0,2,0,-2.6,-7.0
4,2001,AC Green,5,5,2000-11-10,37-037,MIA,@,UTA,W (+7),0,15:08,0,2,0.000,0,0,NaN,0,0,NaN,2,3,5,0,0,1,1,2,0,-0.2,11.0


In [5]:
players_boxscores[players_boxscores['player_name'].str.contains('LeBron James')].head()

,id_season,player_name,rk,game_id,game_date,age,tm,ext_dom,ppp,results,gs,mp,fg,fga,fg_prct%,3p,3pa,3p_prct,ft,fta,ft_prct,orb,drb,trb,ast,stl,blk,tov,pf,pts,player_game_score,plus_minus
358134,2004,LeBron James,1,1,2003-10-29,18-303,CLE,@,SAC,L (-14),1,42:50,12,20,0.600,0,2,0.0,1,3,0.333,2,4,6,9,4,0,2,3,25,24.7,-9.0
358135,2004,LeBron James,2,2,2003-10-30,18-304,CLE,@,PHO,L (-9),1,40:21,8,17,0.471,1,5,0.2,4,7,0.571,2,10,12,8,1,0,7,1,21,14.7,-3.0
358136,2004,LeBron James,3,3,2003-11-01,18-306,CLE,@,POR,L (-19),1,39:10,3,12,0.250,0,1,0.0,2,2,1.000,0,4,4,6,2,0,2,3,8,5.0,-21.0
358137,2004,LeBron James,4,4,2003-11-05,18-310,CLE,NaN,DEN,L (-4),1,41:06,3,11,0.273,0,2,0.0,1,1,1.000,2,9,11,7,2,3,2,1,7,11.2,-3.0
358138,2004,LeBron James,5,5,2003-11-07,18-312,CLE,@,IND,L (-1),1,43:44,8,18,0.444,1,2,0.5,6,7,0.857,0,5,5,3,0,0,7,2,23,9.0,-7.0


In [10]:
players_boxscores[players_boxscores['player_name'].str.contains('Dwyane Wade')].head()

,id_season,player_name,rk,game_id,game_date,age,tm,ext_dom,ppp,results,gs,mp,fg,fga,fg_prct%,3p,3pa,3p_prct,ft,fta,ft_prct,orb,drb,trb,ast,stl,blk,tov,pf,pts,player_game_score,plus_minus
174822,2004,Dwyane Wade,1,1,2003-10-28,21-284,MIA,@,PHI,L (-15),1,41:02,8,18,0.444,0,1,0.0,2,3,0.667,1,3,4,4,0,0,5,3,18,6.4,-5.0
174823,2004,Dwyane Wade,2,2,2003-10-29,21-285,MIA,@,BOS,L (-23),1,33:32,2,12,0.167,1,2,0.5,0,0,NaN,1,1,2,4,1,0,0,5,5,0.2,-17.0
174824,2004,Dwyane Wade,3,3,2003-10-31,21-287,MIA,NaN,DET,L (-12),1,23:41,1,9,0.111,0,0,NaN,2,2,1.000,2,1,3,2,0,1,2,1,4,-0.5,-17.0
174825,2004,Dwyane Wade,6,4,2003-11-08,21-295,MIA,NaN,MIN,L (-9),1,42:00,8,21,0.381,1,2,0.5,2,4,0.500,0,3,3,5,1,1,3,3,19,8.6,-6.0
174826,2004,Dwyane Wade,7,5,2003-11-11,21-298,MIA,@,HOU,L (-20),1,37:54,4,9,0.444,0,1,0.0,3,4,0.750,0,5,5,5,3,0,3,3,11,9.7,-15.0


# ----------------------------------------------------------------------------
# PART 2: Players Attributes
# ----------------------------------------------------------------------------

In [6]:
players_attributes = pd.read_csv('./pipeline_output/nba_list_players_multi_season_dataset_2022-08-31.csv')
players_attributes.shape

(11952, 10)

In [7]:
players_attributes.head()

,id_season,tm,Name,Position,Ht,Wt,BirthDate,Nationality,Experience,College
0,2001,ATL,Cal Bowdler,SF,6-10,245,"March 31, 1977",us,1,Old Dominion
1,2001,ATL,Ira Bowman,PG,6-5,195,"June 11, 1973",us,1,"Providence, Penn"
2,2001,ATL,Sean Colson,PG,6-0,175,"July 1, 1975",us,R,"Rhode Island, UNC Charlotte"
3,2001,ATL,Chris Crawford,PF,6-9,235,"May 13, 1975",us,3,Marquette
4,2001,ATL,Dion Glover,SG,6-5,228,"October 22, 1978",us,1,Georgia Tech


In [8]:
pd.to_datetime(players_attributes['BirthDate']).head()

0       1977-03-31
1       1973-06-11
2       1975-07-01
3       1975-05-13
4       1978-10-22
5       1972-12-02
6       1970-10-14
7       1974-10-02
8       1980-05-05
9       1975-11-08
10      1968-09-18
11      1971-12-06
12      1975-11-17
13      1971-10-22
14      1977-09-05
15      1976-09-09
16      1966-06-25
17      1977-11-27
18      1968-01-11
19      1977-05-08
20      1968-06-14
21      1977-09-15
22      1975-11-04
23      1970-10-09
24      1976-02-11
25      1975-11-30
26      1968-05-22
27      1972-06-14
28      1974-03-12
29      1974-07-04
30      1975-09-27
31      1974-02-01
32      1978-06-15
33      1969-08-03
34      1978-02-07
35      1977-10-13
36      1975-03-21
37      1970-12-10
38      1976-08-12
39      1972-07-17
40      1969-10-14
41      1971-01-12
42      1968-07-23
43      1967-06-21
44      1979-04-13
45      1966-09-29
46      1976-12-26
47      1978-05-21
48      1972-11-29
49      1975-02-22
50      1969-08-03
51      1967-12-14
52      1976

In [ ]:

# ####################################################################################################################
# #
# #
# # FEATURE ENGINEERING:
# #
# #
# ####################################################################################################################

# #-----------------------------------------
# # wt - NOTHING TO DO - WEIGHT EXPRESS IN LBS
# #-----------------------------------------

# data_player_info_and_salary_full$wt <- as.numeric(data_player_info_and_salary_full$wt)
# summary(data_player_info_and_salary_full$wt)

# #-----------------------------------------
# # birth_date
# #-----------------------------------------

# # FROMAT DATE = January 14, 1985 SO WE CONVERT TO as.Date
# data_player_info_and_salary_full$birth_date_2 <- as.Date(data_player_info_and_salary_full$birth_date, "%B %d, %Y")

# # TO HAVE THE AGE AT THE START OF THE SEASON WE HAVE TO GET THE DATE FOR THE START SEASON FOR EACH YEAR

# mydb <- dbConnect(MySQL(),
#                   user = 'root',
#                   password = 'P21yfuobi',
#                   dbname= "nba",
#                   host = '127.0.0.1',
#                   port = 3306)


# nba_game_data <- as.data.frame((dbGetQuery(mydb, "SELECT * FROM nba_game_data")))
# data_min_season <- nba_game_data %>% group_by(id_season) %>% summarise(date_start_season = min(as.Date(game_date)))

# data_player_info_and_salary_full <- merge(data_player_info_and_salary_full,
#                                           data_min_season,
#                                           by.x = "year",
#                                           by.y = "id_season",
#                                           all.x = TRUE)



# # GET THE YEAR DIFFERENCE
# data_player_info_and_salary_full$age <- round(as.numeric(difftime(data_player_info_and_salary_full$date_start_season,
#                                                                   data_player_info_and_salary_full$birth_date_2,
#                                                                   unit="weeks"))/52.25,1)



# #-----------------------------------------
# # exp
# #-----------------------------------------

# # WE SET THE FORMAT AS NUMERIC AND RECODE THE R EQUAL TO 0
# data_player_info_and_salary_full$exp <- as.numeric(ifelse(data_player_info_and_salary_full$exp  == "R",
#                                                           0,
#                                                           data_player_info_and_salary_full$exp ))

# #-----------------------------------------
# # ht - 6-5 to 185 cm
# #-----------------------------------------

# # STEP 1 : ISOLATE FOOT WICH IS THE FIRST NUMBER  THEN ISOLATE INCH WICH IS THE SECOND NUMBER
# # 1 inche = 2.54cm
# # 1 foot = 30.48cm

# library(tidyr)
# data_player_info_and_salary_full <- data_player_info_and_salary_full %>% separate(ht,
#                                                                                   c("foot", "inch" ),
#                                                                                   "-")


# data_player_info_and_salary_full$foot <- as.numeric(data_player_info_and_salary_full$foot)
# data_player_info_and_salary_full$inch <- as.numeric(data_player_info_and_salary_full$inch)

# data_player_info_and_salary_full$cm_size <- data_player_info_and_salary_full$foot * 30.48 + data_player_info_and_salary_full$inch * 2.54

# #-----------------------------------------
# # MERGE SALARY NA FILES PROVIDE BY ANTHONYC AND FLORIANC
# #-----------------------------------------


# #-----------------------------------------
# # EXPRESS SALARY IN MILLION
# #-----------------------------------------

# data_player_info_and_salary_full$salary <- data_player_info_and_salary_full$salary / 10^6

# #-----------------------------------------
# # SELECT THE RELEVATN VARAIBLES
# #-----------------------------------------

# # WITH DID THE RECODE TO ASSURE THE DATA MANAGEMENT FOR ALL THE TAB
# data_player_info_and_salary_full$id_season <- data_player_info_and_salary_full$year 

# data_player_info_and_salary_full <- data_player_info_and_salary_full %>% select(player,
#                                                                                 tm,
#                                                                                 id_season,
#                                                                                 pos,
#                                                                                 country,
#                                                                                 college,
#                                                                                 age,
#                                                                                 exp,
#                                                                                 cm_size,
#                                                                                 wt,
#                                                                                 salary)

# #--------------------------------------------------------------------------------
# # SAVE THE FILES
# #-------------------------------------------------------------------------------

# write.csv(data_player_info_and_salary_full,
#           paste0("NBA DEV/NBA PLAYER INFORMATION/data_player_info_and_salary_full_prod.csv"),row.names=FALSE)